In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import numpy as np
from custom_dataset import CustomDataset, ResidualBlock, ResNet,CustomResNet50, GCN

import plotly.graph_objects as go
from torch_geometric.nn import MLP
from pytorch_metric_learning.losses import NTXentLoss
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
MUSIC_DIRECTORY = 'Music'# Sets the music directory variable to 'MUSIC' which is a folder that stores spectograms

In [10]:
# Initialize an empty list to store the paths of PNG files
file_paths = []

# Initialize an empty list to keep track of already processed files
processed_files = []

# Walk through the 'Music' directory
for directory, _, files in os.walk('Music'):
    # Filter out files that end with '.png' and have not been processed yet
    png_files = [f for f in files if f.endswith('.png') and f not in processed_files]
    
    # If there are any new PNG files found
    if png_files:
        # Add the full paths of the new PNG files to the file_paths list
        file_paths += [os.path.join(directory, f) for f in png_files]
        
        # Add the new PNG files to the processed_files list to avoid duplicates
        processed_files += png_files


In [11]:
# Create an instance of the CustomDataset class with the music_directory set to MUSIC_DIRECTORY
custom_dataset = CustomDataset(music_directory=MUSIC_DIRECTORY)

In [13]:
len(custom_dataset)

300

In [12]:
# Create a DataLoader instance for the custom_dataset
# Set the batch size to 30 and enable shuffling of the data
train_loader = DataLoader(custom_dataset, batch_size=30, shuffle=True)


In [15]:
# Determine the device to use for PyTorch operations
# Use CUDA if it is available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
# Create an instance of the ResNet model with ResidualBlock and specified layers
# Move the model to the specified device (either CUDA or CPU)
model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

In [17]:
# Initialize the loss function with a specified temperature parameter
loss_func = NTXentLoss(temperature=0.1)

# Initialize the Adam optimizer with the parameters of the model
optimizer = torch.optim.Adam(model.parameters())

# Initialize the learning rate scheduler with the optimizer, 
# set the step size to 10 and the gamma (decay factor) to 0.5
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10, gamma=0.5)


In [18]:
# Define the training function
def train():
    model.train()  # Set the model to training mode
    total_loss = 0  # Initialize the total loss
    total_pairs = 0  # Initialize to keep track of the total number of pairs processed
    outputs_cnn_list = []

    # Iterate through the DataLoader
    for batch_idx, (patch1, patch2, index, filename) in enumerate(tqdm(train_loader)):
        patch1 = patch1.to(device)  # Move the first patch to the specified device
        patch2 = patch2.to(device)  # Move the second patch to the specified device

        optimizer.zero_grad()  # Zero the gradients

        # Forward pass through the model
        #Output1 is the outpout after it has gone through the mlp and output1_cnn is the 512 channel cnn output 
        output_1, output_2, output1_cnn, output2_cnn = model(patch1, patch2)

        # Prepare embeddings for loss computation
        embeddings = torch.cat((output_1, output_2))  # Concatenate the output embeddings
        batch_size = patch1.size(0)  # Get the batch size
        indices = torch.arange(0, batch_size, device=device)  # Create a tensor of indices
        labels = torch.cat((indices, indices))  # Create the labels for NTXentLoss
        loss = loss_func(embeddings, labels)  # Compute the loss

        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters

        total_loss += loss.item() * batch_size  # Accumulate the loss
        total_pairs += batch_size  # Update the total number of pairs processed

        # Convert output1_cnn to numpy and store in list
        outputs_cnn_np = output1_cnn.detach().cpu().numpy()
        outputs_cnn_list.append(outputs_cnn_np)

    return total_loss / total_pairs, outputs_cnn_list  # Return the average loss per pair


In [ ]:
# Initialize an empty list to store loss values for each epoch
loss_values = []

# Iterate through 200 epochs
for epoch in tqdm(range(200)):
    # Call the train function and get the loss for the current epoch
    loss,outputs_cnn_list = train()
    
    # Print the epoch number and the corresponding loss value
    print(f"Epoch {epoch:03d}, Loss {loss:.4f}")
    
    # Append the loss value to the loss_values list
    loss_values.append(loss)
    
    # Step the learning rate scheduler
    scheduler.step()


In [ ]:
# Concatenate the list of CNN outputs along the first axis
outputs_cnn_array = np.concatenate(outputs_cnn_list, axis=0)

# Save the concatenated CNN outputs array to a numpy file
np.save("outputs_cnn.npy", outputs_cnn_array)


In [ ]:
cnn_values=np.load("outputs_cnn.npy")

In [ ]:

model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)

# Load the model checkpoint from the specified file
checkpoint = torch.load("model.pth")

# Load the state dictionary from the checkpoint into the model
model.load_state_dict(checkpoint)


In [ ]:
model.eval()  # Set the model to evaluation mode
all_outputs = []
all_labels = []

with torch.no_grad():  # Disable gradient computation
    for input1, input2, labels_index, labels_name  in train_loader:
        input1 = input1.to(device)
        input2 = input2.to(device)
        _, _, output1_cnn, _ = model(input1, input2)
        output1_cnn = output1_cnn.cpu().detach()

        all_outputs.append(output1_cnn)
        all_labels.append(labels_index.cpu())

# Concatenate all collected outputs and labels
all_outputs = torch.cat(all_outputs).squeeze()
all_labels = torch.cat(all_labels).squeeze().numpy()  #